In [10]:
import os

os.environ['OPENAI_API_KEY']=''
os.environ['LANGCHAIN_TRACING_V2']=''
os.environ['LANGCHAIN_ENDPOINT']=''
os.environ['LANGCHAIN_API_KEY']=''
os.environ['LANGCHAIN_PROJECT']=''

In [11]:
from langchain_openai import OpenAIEmbeddings

# OpenAI의 "text-embedding-3-large" 모델을 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [12]:
text = "임베딩 테스트를 하기 위한 샘플 문장입니다."

In [13]:
# 텍스트를 임베딩하여 쿼리 결과를 생성합니다.
query_result = embeddings.embed_query(text)

### Document Embedding

In [14]:
text_2 = '임베딩(Embedding)이란 데이터를 고차원에서 저차원 벡터로 변환해 컴퓨터가 효율적으로 이해하고 처리할 수 있도록 표현하는 방법'

doc_result = embeddings.embed_documents(
    [text_2, text_2, text_2, text_2]
) # 텍스트를 임베딩하여 문서 벡터를 생성

In [15]:
doc_result[0][:5]

[-0.00523593183606863,
 -0.006059447303414345,
 0.00289501273073256,
 0.02293643169105053,
 0.025295142084360123]

In [16]:
len(doc_result[0])

1536

### 차원을 강제 배정하는 방법

In [24]:
# OpenAI의 "text-embedding-3-small" 모델을 사용하여 1024차원의 임베딩을 생성하는 객체를 초기화합니다.
scaled_embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1024)

In [25]:
scaled_embeddings.embed_documents([text])

[[-0.008802819065749645,
  0.04173462465405464,
  0.022164570167660713,
  -0.02230047434568405,
  0.019508281722664833,
  -0.009500866755843163,
  -0.013145542703568935,
  0.0008602052694186568,
  0.031924888491630554,
  -0.04642946273088455,
  0.01520879939198494,
  0.06335558742284775,
  -0.002988325199112296,
  0.02328886091709137,
  -0.014195702970027924,
  -0.04195701330900192,
  -0.020842604339122772,
  -0.008222141303122044,
  0.017469733953475952,
  0.02218928001821041,
  0.01774154044687748,
  -0.0023520514369010925,
  -0.0006598252803087234,
  0.013886832632124424,
  -0.0009289290173910558,
  -0.060390423983335495,
  0.004450828768312931,
  0.014603412710130215,
  0.052186816930770874,
  -0.036397337913513184,
  -0.006356561556458473,
  -0.020805539563298225,
  0.026019278913736343,
  -0.029948115348815918,
  0.060983456671237946,
  0.033358048647642136,
  -0.03308624401688576,
  -0.010829010978341103,
  0.004271683748811483,
  0.007795899640768766,
  -0.0010393502889201045,


In [27]:
len(scaled_embeddings.embed_documents([text])[0])

1024

### Similarity Cals.

In [28]:
sentence1 = "안녕하세요? 반갑습니다."
sentence2 = "안녕하세요? 반갑습니다!"
sentence3 = "안녕하세요? 만나서 반가워요."
sentence4 = "Hi, nice to meet you."
sentence5 = "I like to eat apples."

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

sentences = [sentence1, sentence2, sentence3, sentence4, sentence5]
embedded_sentences = scaled_embeddings.embed_documents(sentences)

In [33]:
len(embedded_sentences[0])

1024

In [34]:
def similarity(a, b):
    return cosine_similarity([a], [b])[0][0]

In [35]:
# embedding 할때 언어가 달라지면 유사도 현저히 낮아짐

for i, sentence in enumerate(embedded_sentences):
    for j, other_sentence in enumerate(embedded_sentences):
        if i < j:
            print(
                f'[유사도 {similarity(sentence, other_sentence):.4f}] {sentences[i]} \t <====> \t {sentences[j]}'
            )

[유사도 0.9644] 안녕하세요? 반갑습니다. 	 <====> 	 안녕하세요? 반갑습니다!
[유사도 0.8376] 안녕하세요? 반갑습니다. 	 <====> 	 안녕하세요? 만나서 반가워요.
[유사도 0.5042] 안녕하세요? 반갑습니다. 	 <====> 	 Hi, nice to meet you.
[유사도 0.1362] 안녕하세요? 반갑습니다. 	 <====> 	 I like to eat apples.
[유사도 0.8142] 안녕하세요? 반갑습니다! 	 <====> 	 안녕하세요? 만나서 반가워요.
[유사도 0.4790] 안녕하세요? 반갑습니다! 	 <====> 	 Hi, nice to meet you.
[유사도 0.1318] 안녕하세요? 반갑습니다! 	 <====> 	 I like to eat apples.
[유사도 0.5128] 안녕하세요? 만나서 반가워요. 	 <====> 	 Hi, nice to meet you.
[유사도 0.1409] 안녕하세요? 만나서 반가워요. 	 <====> 	 I like to eat apples.
[유사도 0.2249] Hi, nice to meet you. 	 <====> 	 I like to eat apples.


In [ ]:
# 뜻은 같지만, 언어가 달라 유사도가 떨어지는 경우 해결 방법

